In [5]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.decomposition import TruncatedSVD
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline, make_pipeline
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv("data/MBTI_500.csv")
data['I/E'] = data['type'].str[0]
data.loc[((data['I/E'] == 'I') & (data['type'].str[3] == 'J')) | ((data['I/E'] == 'E') & (data['type'].str[3] == 'P')), 'dominant_auxiliary'] = data.loc[((data['I/E'] == 'I') & (data['type'].str[3] == 'J')) | ((data['I/E'] == 'E') & (data['type'].str[3] == 'P')), "type"].str[1:3]
data.loc[data['dominant_auxiliary'].isnull(), 'dominant_auxiliary'] = data.loc[data['dominant_auxiliary'].isnull(), "type"].str[-2:-4:-1]
data

,posts,type,I/E,dominant_auxiliary
0,know intj tool use interaction people excuse a...,INTJ,I,NT
1,rap music ehh opp yeah know valid well know fa...,INTJ,I,NT
2,preferably p hd low except wew lad video p min...,INTJ,I,NT
3,drink like wish could drink red wine give head...,INTJ,I,NT
4,space program ah bad deal meing freelance max ...,INTJ,I,NT
...,...,...,...,...
106062,stay frustrate world life want take long nap w...,INFP,I,FN
106063,fizzle around time mention sure mistake thing ...,INFP,I,FN
106064,schedule modify hey w intp strong wing underst...,INFP,I,FN
106065,enfj since january busy schedule able spend li...,INFP,I,FN


In [3]:
vectorizer = TfidfVectorizer()
x_tfidf = vectorizer.fit_transform(data['posts'])

In [6]:
#Perform undersampling
under = RandomUnderSampler(sampling_strategy = dict(np.minimum(data['type'].value_counts(), 2000)))
x, y = under.fit_resample(x_tfidf, data['type'])

#Dimesionality reduction
svd = TruncatedSVD(n_components=250)
x = svd.fit_transform(x)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.4)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.75)

#Oversampling
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)
x_val, y_val = smote.fit_resample(x_val, y_val)
x_test, y_test = smote.fit_resample(x_test, y_test)
y_train = pd.get_dummies(y_train)
y_val = pd.get_dummies(y_val)
n_cols = x_train.shape[1]

In [ ]:
y_train

In [7]:
#Initialise - 1 layer with 500 nodes
model = Sequential()
model.add(Dropout(rate=0.2))
model.add(Dense(500, activation='relu', input_shape=(n_cols, )))
model.add(Dropout(rate=0.2))
model.add(Dense(500, activation='relu', input_shape=(n_cols, )))
model.add(Dense(16, activation='softmax'))

#Compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

#Run model with early stopping
early_stopping_monitor = EarlyStopping(patience=2)
model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stopping_monitor])

y_pred = model.predict(x_test)

#For each of the testing data, find the type with highest probability
pred_types = []
types = y_train.columns
for i in range(len(y_pred)):
    pred_types.append(types[y_pred[i].argmax()])

Epoch 1/100
612/612 [==============================] - 4s 5ms/step - loss: 2.7692 - accuracy: 0.0991 - val_loss: 2.7618 - val_accuracy: 0.1910
Epoch 2/100
612/612 [==============================] - 3s 5ms/step - loss: 2.7570 - accuracy: 0.1926 - val_loss: 2.7489 - val_accuracy: 0.3267
Epoch 3/100
612/612 [==============================] - 3s 5ms/step - loss: 2.7439 - accuracy: 0.2679 - val_loss: 2.7335 - val_accuracy: 0.4035
Epoch 4/100
612/612 [==============================] - 3s 5ms/step - loss: 2.7273 - accuracy: 0.2994 - val_loss: 2.7133 - val_accuracy: 0.4826
Epoch 5/100
612/612 [==============================] - 3s 5ms/step - loss: 2.7045 - accuracy: 0.3324 - val_loss: 2.6857 - val_accuracy: 0.4689
Epoch 6/100
612/612 [==============================] - 4s 6ms/step - loss: 2.6743 - accuracy: 0.3458 - val_loss: 2.6485 - val_accuracy: 0.5328
Epoch 7/100
612/612 [==============================] - 4s 6ms/step - loss: 2.6341 - accuracy: 0.3660 - val_loss: 2.6013 - val_accuracy: 0.4988

Epoch 58/100
612/612 [==============================] - 5s 8ms/step - loss: 0.7800 - accuracy: 0.7729 - val_loss: 0.6617 - val_accuracy: 0.8247
Epoch 59/100
612/612 [==============================] - 5s 8ms/step - loss: 0.7686 - accuracy: 0.7792 - val_loss: 0.6521 - val_accuracy: 0.8250
Epoch 60/100
612/612 [==============================] - 4s 7ms/step - loss: 0.7666 - accuracy: 0.7772 - val_loss: 0.6561 - val_accuracy: 0.8233
Epoch 61/100
612/612 [==============================] - 4s 7ms/step - loss: 0.7531 - accuracy: 0.7827 - val_loss: 0.6497 - val_accuracy: 0.8241
Epoch 62/100
612/612 [==============================] - 4s 7ms/step - loss: 0.7552 - accuracy: 0.7851 - val_loss: 0.6490 - val_accuracy: 0.8233
Epoch 63/100
612/612 [==============================] - 4s 7ms/step - loss: 0.7485 - accuracy: 0.7837 - val_loss: 0.6567 - val_accuracy: 0.8201
Epoch 64/100
612/612 [==============================] - 4s 7ms/step - loss: 0.7551 - accuracy: 0.7808 - val_loss: 0.6523 - val_accuracy:

In [ ]:
print(classification_report(y_test, pred_types))

In [ ]:
print("Accuracy =", accuracy_score(y_test, pred_types))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

import opendatasets as od

In [ ]:
#remove mbti-related text, so we dont classify a type based on how much some mbti string appears inside (not generalised)
#for loop creates all stuff like XXTJ etc, cause some posts contains thing like this

mbti_types = ['ISTJ', 'ISTP', 'ISFJ', 'ISFP', 'INTJ', 'INTP', 'INFJ', 'INFP', 'ESTJ', 'ESTP', 'ESFJ', 'ESFP', 'ENTJ', 'ENTP', 'ENFJ', 'ENFP']
mbti_stopwords = ['fe', 'fi', 'ne', 'ni', 'se', 'si', 'te', 'ti']
for i in mbti_types:
  for mask in range(17):
    tmp = list(i)
    for j in range(4):
      if pow(2, j) & mask != 0:
        tmp[j] = 'X'
    mbti_stopwords.append("".join(tmp))
    mbti_stopwords.append("".join(tmp) + "s")